In [5]:
# Install required libraries (if needed)
!pip install pandas scikit-learn openpyxl matplotlib keras_tuner

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import keras_tuner as kt
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_excel('./data/dataset.xlsx', sheet_name='Dataset')
data.columns = data.columns.str.strip()

# Select relevant columns
features = ['INFLOW', 'EVAPORATION', 'RESERVOIR STORAGE', 'RESERVOIR LEVEL']
target = 'RELEASE (OUTFLOW)'
X = data[features]
y = data[target]

# Standardize features
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

# Normalize target variable
target_scaler = MinMaxScaler()
y_scaled = target_scaler.fit_transform(y.values.reshape(-1, 1))

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(hp.Int('units_1', min_value=32, max_value=256, step=32), activation='relu', input_dim=X_train.shape[1]))
    
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(hp.Int(f'units_{i+2}', min_value=32, max_value=128, step=32), activation='relu'))
    
    model.add(layers.Dense(1, activation='linear'))
    
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                  loss='mse', metrics=['mae'])
    return model

# Hyperparameter tuning
tuner = kt.Hyperband(build_model, objective='val_loss', max_epochs=50, factor=3, directory='tuner_dir', project_name='ann_tuning')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Run the hyperparameter search
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[early_stopping], verbose=1)

# Get the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters: {best_hps.values}")

best_model = tuner.hypermodel.build(best_hps)

# Train best model
history = best_model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2, verbose=1, callbacks=[early_stopping])

# Predict and inverse transform
y_pred_scaled = best_model.predict(X_test)
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_orig = target_scaler.inverse_transform(y_test)

# Evaluate
mse = mean_squared_error(y_test_orig, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_orig, y_pred)
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")

# Plot loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()


ModuleNotFoundError: No module named 'keras_tuner'